In [31]:
import os
import uuid
import numpy as np
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
from flask_cors import CORS
from jinja2 import TemplateNotFound 

# Path to the public folder
base_path = r'./public/'

# Initialize Flask application
app = Flask(__name__, static_folder="public")
cors = CORS(app, origins=["http://localhost:5000"])

# Load the TensorFlow model
model_path = r"mnist-cnn.keras"


# Endpoint for digit recognition from uploaded image
@app.route('/', methods=['POST', 'GET'])
def index():
    try:
        return render_template(r'\public\default.html')
    except TemplateNotFound:
        return "Template not found!", 404

@app.route('/process', methods=['POST'])
def upload_file():
    try:
        # Get the file from the POST request
        file = request.files['file']

        # Generate a UUID identifier
        c_uuid = str(uuid.uuid4())

        # Get the file extension
        file_ext = file.filename.rsplit('.', 1)[1].lower()

        # Construct the file path
        filename = os.path.join(base_path, c_uuid + '.' + file_ext)

        # Save the file
        file.save(filename)

        # Load the image and convert to grayscale
        img = Image.open(filename).convert("L")

        # Resize the image
        new_image = img.resize((28, 28))

        # Convert image to array and resize
        x = np.array(new_image).reshape((28, 28, 1))
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])

        # Predict the digit in the image
        classes = model.predict(images, batch_size=1)

        # Get the predicted digit (class)
        result = int(np.argmax(classes))

        # Close the image
        img.close()

        # Remove the uploaded image
        os.remove(filename)

        # Return the result as JSON
        return jsonify({'value': result})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
    model = tf.keras.models.load_model(model_path)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.106:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jul/2024 20:10:20] "POST /process HTTP/1.1" 500 -
127.0.0.1 - - [02/Jul/2024 20:10:22] "POST /process HTTP/1.1" 500 -
